In [1]:
import nltk
# nltk.download('punkt', download_dir='NLTK_DATA')
# nltk.download('stopwords', download_dir='NLTK_DATA')
nltk.data.path.append("NLTK_DATA")
import pandas as pd
import numpy as np
from processing import *
from sentiment2 import *

# PREPARASI *TRAINING DATASET*

In [2]:
# TODO: pindah ke sqlite
dataset = pd.read_csv("dataset/training_dataset.csv")
all_pos = dataset[dataset['label'] == 1]
all_neg = dataset[dataset['label'] == 0]

# * Training data: 80% pertama
# * Testing data: 20% terakhir
train_pos = all_pos[:int(len(all_pos)*0.8)]
train_neg = all_neg[:int(len(all_neg)*0.8)]
test_pos = all_pos[-int(len(all_pos)*0.2):]
test_neg = all_neg[-int(len(all_neg)*0.2):]

train_x = train_pos.append(train_neg, ignore_index=True)
test_x = test_pos.append(test_neg, ignore_index=True)

train_y = np.append(np.ones(len(train_pos)), np.zeros(len(train_neg)))
test_y = np.append(np.ones(len(test_pos)), np.zeros(len(test_neg)))

In [3]:
ld = len(dataset)
trp = len(train_pos)
tng = len(train_neg)
tsp = len(test_pos)
tsn = len(test_neg)
print(f"dataset: {ld} \n train pos: {trp} \n train neg: {tng} \n test pos: {tsp} \n test neg: {tsn}")

dataset: 1300 
 train pos: 511 
 train neg: 528 
 test pos: 127 
 test neg: 132


# PELATIHAN *MODEL CLASSIFIER*

In [4]:
x = train_x['text']
y = train_y
kamus_frek = kamus_freq(x, y)

In [5]:
logprior, loglikelihood = train_nbc(kamus_frek, train_x, train_y)

In [6]:
# file = open("/home/gaz/dev/flask-sk/database/prior.txt", "w")
# str = repr(logprior)
# file.write(str)
# file.close()

In [7]:
# like = pd.DataFrame(list(loglikelihood.items()), columns = ['kata', 'likelihood'])
# like.to_csv('/home/gaz/dev/flask-sk/database/loglikelihood.csv')

In [8]:
xkamus = kamus_freq(dataset['text'], dataset['label'])

In [9]:
xprior, xlikelihood = train_nbc(xkamus, dataset['text'], dataset['label'])

In [10]:
xtest = test_nbc(dataset['text'], dataset['label'], xprior, xlikelihood)
xtest

0.9815384615384616

## HASIL PENGUJIAN MODEL
- data *training*
- data *testing*

In [11]:
hasil_uji = test_nbc(x, y, logprior, loglikelihood)
hasil_uji

KeyboardInterrupt: 

In [ ]:
test = test_nbc(test_x['text'], test_y, logprior, loglikelihood)
test

## UJI KESELURUHAN *DATASET*

In [ ]:
data_test = test_nbc(dataset['text'], dataset['label'], logprior, loglikelihood)
data_test

## *CONFUSION MATRIX*

In [ ]:
pdc = pd.DataFrame()
pdc = dataset
pdc['label_prediksi'] = pdc['text'].apply(lambda x: predict_nbc(x, logprior, loglikelihood))

In [ ]:
def toLabel(x):
    if x > 0:
        return 1
    elif x < 0:
        return 0
    else:
        return "n"
pdc['prediksi'] = pdc['label_prediksi'].apply(lambda x: toLabel(x))

In [ ]:
pdc

In [ ]:
conf_matrix = pd.crosstab(pdc['label'], pdc['prediksi'], rownames=['Actual'], colnames=['Predicted'])
conf_matrix

In [ ]:
tp = 558
tn = 647
fp = 14
fn = 81
pop = tp+tn+fp+fn
acc = (tp+tn)/pop
mis = (fp+fn)/pop
pre = tp/(tp+fp)
sen = tp/(tp+fn)
print(f"akurasi: {acc}\nmis-klasifikasi: {mis}\npresisi: {pre}\nsensitivitas: {sen}")

In [ ]:
import seaborn as sn
import matplotlib.pyplot as plt

sn.heatmap(conf_matrix, annot=True, fmt='g')
plt.show()

# UJI CLASSIFIER PADA ARTIKEL BERITA

In [ ]:
berita = open('berita.txt', 'r')
berita = berita.read()

## SENTIMEN ARTIKEL KESELURUHAN

In [ ]:
predict_nbc(berita, logprior, loglikelihood)

## SENTIMEN ARTIKEL PER_KALIMAT

In [ ]:
from nltk.tokenize import sent_tokenize
x = pd.DataFrame()
kalimat = sent_tokenize(berita)
x['kalimat'] = kalimat
x['kalimat_normal'] = x['kalimat'].apply(lambda x: normalisasi(x))

In [ ]:
x['label_prediksi'] = x['kalimat'].apply(lambda x: predict_nbc(x, logprior, loglikelihood))

In [ ]:
x

## SENTIMEN ARTIKEL PER_KATA

In [ ]:
berita_normal = normalisasi(berita)

In [ ]:
# * predict_nbc per-kata
def predict_perkata(text, logpri, loglik):
    l = []
    for w in text:
        x = predict_nbc(w, logpri, loglik)
        l.append({
            w,
            x
            })
    return l

In [ ]:
predict_perkata(berita_normal, logprior, loglikelihood)